# WPILib ML Notebook


## Introduction

By using this notebook, you can train a TensorFlow Lite model for use on a Raspberry Pi and Google Coral USB Accelerator. We've designed this process to be as simple as possible. If you find an issue with this notebook, please create a new issue report on our GitHub page, where you downloaded this notebook.

## Workflow

This section will explain the four distinct steps to getting a trained model to run on your hardware.
### Getting Data

WPILib provides thousands of labelled images for this years game, which you can download here. However, you can train with custom data using this notebook as well. The below instructions describe how to gather and label your own data.

1. Plug a USB Camera into your laptop, and run a script similar to record_video.py, which simply makes an mp4 from the camera stream.
2. Create a [supervise.ly](supervise.ly) account. This is a very nice tool for labelling data.
3. (Optional) You can add other teammates to your Supervise.ly workspace by clicking 'Members' on the left and then 'INVITE' at the top.
4. Choose a workspace to work in, in the 'Workspaces' tab.
5. Upload the official WPILib labelled data to your workspace. Download the tar here, extract it, then click 'IMPORT DATA' or 'UPLOAD' inside of your workspace. Change the import plugin to Supervisely, then drag in the extracted FOLDER. Then, give the project a name, then click import.
6. Upload your own video to your workspace. Click 'UPLOAD' when inside of your workspace, change your import plugin to video, drag in your video, give the project a name, and click import.
7. Click into your newly import Dataset. Use the rectangle tool to draw appropriate boxes around the objects which you wish to label.

### Training

1. Download your datasets from Supervise.ly. Select the "json and jpeg" option.
2. Upload your tar to a new folder in an Amazon S3 bucket, or a brand new S3 bucket.
3. Create a new SageMaker notebook instance, and open the WPILib notebook.
4. Change estimator.fit() to use your new dataset, by specifying the folder in which the tar is stored.
5. Run each block of the notebook in order.
6. Training should take roughly 45 minutes. If you do not change anything in the notebook, it should absolutely not take longer than an hour. In the Training Job, CPU usage should be around 690 during the majority of the training, if running on an ml.c4.2xlarge. If it is less, something went wrong.

### Inference

1. Go to the training job in SageMaker, scroll to the bottom, and find the output S3 location
2. Download the the tar file in the bucket, extract it, and get your .tflite file
3. Put the tflite on your Raspberry Pi by plugging in the SD card into your computer and dragging it in to /home/pi
4. Run the python script, using `python3 object_detection.py --model output.tflite`


## Notebook
### Building and registering the container

This code block runs a script that builds a docker container, and saves it as an Amazon ECR image. This image is used by the training instance so that all proper dependencies and WPILib files are in place.

Just run this one, don't change anything.

In [8]:
%%sh
#!/usr/bin/env bash
docker system prune --force > /dev/null 2>&1

# The name of our algorithm
algorithm_name=sagemaker-tf-wpi

cd container

chmod -R +x coral/

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email) 2> /dev/null

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} . --no-cache > /dev/null 2>&1
docker tag ${algorithm_name} ${fullname} > /dev/null 2>&1

docker push ${fullname} > /dev/null 2>&1


Login Succeeded


### Get execution role

This gets the notebook instance's execution role, used for communicating with the training instance.

Just run this one too.

In [9]:
from sagemaker import get_execution_role

role = get_execution_role()

### Training on SageMaker
Training a model on SageMaker with the Python SDK is done in a way that is similar to the way we trained it locally. This is done by changing our train_instance_type from `local` to one of our [supported EC2 instance types](https://aws.amazon.com/sagemaker/pricing/instance-types/).

In addition, we must now specify the ECR image URL, which we just pushed above.

Just run this one too.

In [10]:
import boto3

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

my_session = boto3.session.Session()
region = my_session.region_name


# If you want to change the algorithm name, make sure to change the name in the first block too.
algorithm_name = 'sagemaker-tf-wpi'

ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

print(ecr_image)

249838237784.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tf-wpi:latest


This last step runs the training instance (default an ml.c4.2xlarge), and begins training with the data specified in `fit()`

This section has lots of values worth changing
`instance_type`: change the type of computer that the training will run on.
`hyperparameters`: change the number of epochs.
`estimator.fit(...)`: change the location of the data used for training. should be in the format `"s3://BUCKET-NAME"`


In [ ]:
from sagemaker.estimator import Estimator

# The type of computer used for training. The default is recommended. It costs 45 cents/hour to run.
instance_type = 'ml.c4.2xlarge'

# The number of epochs to train to. 500 is a safe number. With the default instance, it should take 45 minutes.
hyperparameters = {'epochs': 2000}

# The estimator object, using our notebook, training instance, the ECR image, and the specified training steps
estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name=ecr_image,
                      hyperparameters=hyperparameters)

# Change this bucket if you want to train with your own data. The WPILib bucket contains thousands of high quality labeled images.
# s3://wpilib
estimator.fit("s3://wpilib")


2019-09-19 00:22:09 Starting - Starting the training job...
2019-09-19 00:22:12 Starting - Launching requested ML instances......
2019-09-19 00:23:13 Starting - Preparing the instances for training...
2019-09-19 00:24:10 Downloading - Downloading input data
2019-09-19 00:24:10 Training - Downloading the training image.........
2019-09-19 00:25:36 Training - Training image download completed. Training in progress..+ set -e
+ mkdir /tensorflow/models/research/learn
+ mkdir /tensorflow/models/research/learn/ckpt
+ rm -rf /tensorflow/models/research/learn/train
+ rm -rf /tensorflow/models/research/learn/models
+ cd /tensorflow/models/research
+ echo 'Downloading model'
+ ./prepare_checkpoint_and_dataset.sh --network_type mobilenet_v2_ssd --train_whole_model false
+ network_type=mobilenet_v2_ssd
+ train_whole_model=false
+ [[ 4 -gt 0 ]]
+ case "$1" in
+ network_type=mobilenet_v2_ssd
+ shift 2
+ [[ 2 -gt 0 ]]
+ case "$1" in
+ train_whole_model=false
+ shift 2
+ [[ 0 -gt 0 ]]
+ source /tensor

# The output

Go to the Training Jobs tab of SageMaker. Click on the newest Completed job. Scroll to the bottom. The S3 bucket containing the trained .tflite file (inside of a tar file) can be found there.
